In [1]:
import pymongo
from pymongo import MongoClient
client = MongoClient()
client = MongoClient('localhost', 27017)
# client = MongoClient('mongodb://localhost:27017/')
db = client.db_cdiscount

In [2]:
import io
import bson                       # this is installed with the pymongo package
import matplotlib.pyplot as plt
from skimage.data import imread   # or, whatever image library you prefer
import multiprocessing as mp      # will come in handy due to the size of the data

In [3]:
import numpy as np

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import keras
K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [5]:
first = 0
num_batch = 80
last = 82

In [6]:
epochs = 1
num_classes = db.cat_encode.count()

In [7]:
def simple_model(epochs=epochs, num_classes=num_classes, lrate=0.01, momentum=0.9):
    
    # Convolutional Layer
    model = Sequential()
    # Convolutional Layer
    model.add(Conv2D(180, (3,3), input_shape = (180,180,3), activation='relu'))

    # Pooling Layer
    model.add(MaxPooling2D(pool_size=(1, 1)))

    # Fully conected Layer
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))


    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)

    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [8]:
model =simple_model(epochs=epochs, num_classes=num_classes)

In [14]:
counter = 0
for j in range(0,num_batch-1):
# for j in range(0,1):
    
    a1 = int(list(np.linspace(first,last,num_batch))[j])
    an = int(list(np.linspace(first,last,num_batch))[j+1])
    
    lst_batch = []
    for i in (db.train_example.find({})[a1:an]):
        dic = {}
    #     print('category_id: ', i['category_id'])
        dic['category_id'] = i['category_id']

    #     print('cat: ', db.cat_encode.find_one({ "cat" : (1000004079)}, {"cat" : 1.0, "_id" : 0})['cat'])
        dic['cat'] = db.cat_encode.find_one({ "cat" : (i['category_id'])}, {"cat" : 1.0, "_id" : 0})['cat']

    #     print('img: ', i['imgs'])
#         dic['imgs'] = i['imgs']
        picture = imread(io.BytesIO(i['imgs'][0]['picture']))
        dic['picture'] = picture.reshape(180,180,3)

    #     print('encode: ', db.cat_encode.find_one({ "cat" : (1000004079)}, {"encode" : 1.0, "_id" : 0})['encode'])
        dic['encode'] = db.cat_encode.find_one({ "cat" : (i['category_id'])}, {"encode" : 1.0, "_id" : 0})['encode']

        lst_batch.append(dic)
             
        counter = counter +1
        
    
    X_batch = np.array([lst['picture'] for lst in lst_batch])
    Y_batch = np.array([lst['encode'] for lst in lst_batch])
    print(X_batch.shape)
    print(Y_batch.shape)
    model.fit(X_batch, Y_batch, batch_size=128, epochs=epochs)
    
print(an)

ValueError: cannot reshape array of size 97200 into shape (30,30,3)

In [ ]:
len(np.unique([cat['category_id'] for cat in lst_batch]))

In [ ]:
num_classes=len(dummy_y[81])
epochs = 5